# Project Title
### Data Engineering Capstone Project

For this project ETL i would be using two forms one to analyze my data and the second is to write my findings to script to run the ETL. For this project, I wanted to see if U.S. cities has something to do for people entering. A use case for this would be TSA personal have analytic GUI to see the influx of people coming in, in what city.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, isnull, sum, count
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
import pandas as pd
from pyspark.sql.functions import *
import etl
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

### Step 1: Scope the Project and Gather Data

#### Scope
For this project Im looking into the data to see the tourism within the US.
Im looking into using this data to use analytics table to see a graphical represenation. Going to be using my data with apache spark to better analyze the data for fast querying on huge datasets and saving my results into S3.

#### Describe and Gather Data 
Using the immigration data set and it is from US National Tourism and Trade Office, the information includes detailed information of people entering the us like dates, from what country, and more.
Also will be using Airport Dataframe, because it brings up great information about if airports mean a difference for peoples choice of city. The data is from datahub website.
Final data Ill be using is US demographics, this data simplies the cities people demographics. This data is from opendatasoft.

IMMIGRATION DATAFRAME

In [2]:
immiFname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
dataframe = pd.read_sas(immiFname, 'sas7bdat', encoding="ISO-8859-1")

TEMPERATURE DATAFRAME

In [3]:
tempName = '../../data2/GlobalLandTemperaturesByCity.csv'
tempdf = pd.read_csv(tempName)

AIRPORT DATAFRAME

In [4]:
airportDF = pd.read_csv("/home/workspace/airport-codes_csv.csv")

DEMOGRAPHICS DATAFRAME

In [5]:
demoDF = pd.read_csv("/home/workspace/us-cities-demographics.csv", sep = ";")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

IMMIGRATION DATA

cicid | i94yr | i94mon | i94cit | i94res | i94port | arrdate | i94mode | i94addr | depdate | i94bir | i94visa | count | dtadfile | visapost | occup | entdepa | entdepd | entdepu | matflag | biryear | dtaddto | gender | insnum | airline | admnum | fltno | visatype

In [6]:
dataframe.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [7]:
dataframe.isnull().mean().round(2) * 100

cicid         0.0
i94yr         0.0
i94mon        0.0
i94cit        0.0
i94res        0.0
i94port       0.0
arrdate       0.0
i94mode       0.0
i94addr       5.0
depdate       5.0
i94bir        0.0
i94visa       0.0
count         0.0
dtadfile      0.0
visapost     61.0
occup       100.0
entdepa       0.0
entdepd       4.0
entdepu     100.0
matflag       4.0
biryear       0.0
dtaddto       0.0
gender       13.0
insnum       96.0
airline       3.0
admnum        0.0
fltno         1.0
visatype      0.0
dtype: float64

In [8]:
dataframe = dataframe.drop(columns = ["occup", "entdepu", "insnum"])

In [11]:
dataframe.dropna(subset=['cicid'])
dataframe.drop_duplicates()
print("Number of Distinct rows " + str(dataframe.count()))
dataframe.head()

Number of Distinct rows cicid       3096313
i94yr       3096313
i94mon      3096313
i94cit      3096313
i94res      3096313
i94port     3096313
arrdate     3096313
i94mode     3096074
i94addr     2943941
depdate     2953856
i94bir      3095511
i94visa     3096313
count       3096313
dtadfile    3096312
visapost    1215063
entdepa     3096075
entdepd     2957884
matflag     2957884
biryear     3095511
dtaddto     3095836
gender      2682044
airline     3012686
admnum      3096313
fltno       3076764
visatype    3096313
dtype: int64


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,T,NaN,NaN,1979.0,10282016,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,G,NaN,NaN,1991.0,D/S,M,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,T,O,M,1961.0,09302016,M,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,O,O,M,1988.0,09302016,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,O,O,M,2012.0,09302016,NaN,AA,9.246846e+10,00199,B2


In [12]:
immigrationDF = dataframe

#### Cleaning Immigration Data
I first wanted to see a small picture of the dataframe to see how I want to clean this. I first wanted to see if there was any columns that where null. There were three columns where they had more than 90% null, so I removed those columns. Second, I wanted to drop any duplicates from the dataframe.

Temperature Data

dt | AverageTemperature | AverageTemperatureUncertainty | City | Country | Latitude | Longitude

In [13]:
tempdf.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [14]:
tempdf.isnull().mean().round(2) * 100

dt                               0.0
AverageTemperature               4.0
AverageTemperatureUncertainty    4.0
City                             0.0
Country                          0.0
Latitude                         0.0
Longitude                        0.0
dtype: float64

In [15]:
tempdf.dropna(subset=['dt'])
tempdf.drop_duplicates()

tempdf[["AverageTemperature"]] = tempdf[["AverageTemperature"]].fillna(value = 0.0)
tempdf[["AverageTemperatureUncertainty"]] = tempdf[["AverageTemperatureUncertainty"]].fillna(value = 0.0)
tempdf[["City"]] = tempdf[["City"]].fillna(value = "XX")
tempdf[["AverageTemperatureUncertainty"]] = tempdf[["AverageTemperatureUncertainty"]].fillna(value = 0.0)

print("Number of Distinct rows " + str(tempdf.count()))
tempdf.head()

Number of Distinct rows dt                               8599212
AverageTemperature               8599212
AverageTemperatureUncertainty    8599212
City                             8599212
Country                          8599212
Latitude                         8599212
Longitude                        8599212
dtype: int64


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,0.000,0.000,Århus,Denmark,57.05N,10.33E
2,1744-01-01,0.000,0.000,Århus,Denmark,57.05N,10.33E
3,1744-02-01,0.000,0.000,Århus,Denmark,57.05N,10.33E
4,1744-03-01,0.000,0.000,Århus,Denmark,57.05N,10.33E


#### Temperature Data Cleaning
For Temperature data, I wanted to see how it looks by seeing the dataframe. Like before I went to clean the data and also adding in some dummy variables for nulls. Went to drop a column and removed any duplicates. At the end did not use temperature data.

AIRPORT DATA

ident | type | name | elevation_ft | continent | iso_country | iso_region | municipality | gps_code | iata_code | local_code | coordinates

In [16]:
airportDF.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [17]:
airportDF.isnull().mean().round(2) * 100

ident            0.0
type             0.0
name             0.0
elevation_ft    13.0
continent       50.0
iso_country      0.0
iso_region       0.0
municipality    10.0
gps_code        26.0
iata_code       83.0
local_code      48.0
coordinates      0.0
dtype: float64

In [18]:
airportDF.dropna(subset=['ident', 'coordinates'])
airportDF.drop_duplicates()
airportDF = airportDF.drop(columns = ["iata_code"], axis = 1)

airportDF[["continent"]] = airportDF[["continent"]].fillna(value = "XX")
airportDF[["municipality"]] = airportDF[["municipality"]].fillna(value = "XX")
airportDF[["gps_code"]] = airportDF[["gps_code"]].fillna(value = "XX00")
airportDF[["local_code"]] = airportDF[["local_code"]].fillna(value = "XX00")
print("Number of Distinct rows " + str(airportDF.count()))
airportDF.head()

Number of Distinct rows ident           55075
type            55075
name            55075
elevation_ft    48069
continent       55075
iso_country     54828
iso_region      55075
municipality    55075
gps_code        55075
local_code      55075
coordinates     55075
dtype: int64


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,XX,US,US-PA,Bensalem,00A,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,XX,US,US-KS,Leoti,00AA,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,XX,US,US-AK,Anchor Point,00AK,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,XX,US,US-AL,Harvest,00AL,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,XX,US,US-AR,Newport,XX00,XX00,"-91.254898, 35.6087"


#### Airport Data Cleaning
As before I first clean the data, check to see if there are any columns that have alot of empty. I went to go clean up some rows that where missing some important colums like ident and coordinates. I also removed iata_code, becuase it had 83% of null values in that column. Went to proces the data.

DEMOGRAPHICS DATA

City | State | Median Age | Male Population | Female Population | Total Population | Number of Veterans | Foreign-born | Average Household Size | State Code | Race | Count

In [19]:
demoDF.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [20]:
demoDF.isnull().mean().round(2) * 100

City                      0.0
State                     0.0
Median Age                0.0
Male Population           0.0
Female Population         0.0
Total Population          0.0
Number of Veterans        0.0
Foreign-born              0.0
Average Household Size    1.0
State Code                0.0
Race                      0.0
Count                     0.0
dtype: float64

In [21]:
demoDF.rename(columns = {"Median Age":"medAge", "Male Population":"malePop", "Female Population":"femPop", "Total Population":"totPop", \
                         "Number of Veterans":"numVets", "Foreign-born":"foreignBorn", "Average Household Size":"avgHouseh", \
                         "State Code":"stateCode"}, inplace = True)

demoDF.dropna(subset=['stateCode'])
demoDF.drop_duplicates()

demoDF[["avgHouseh"]] = demoDF[["avgHouseh"]].fillna(value = 0.0)
print("Number of Distinct rows " + str(demoDF.count()))
demoDF.head()

Number of Distinct rows City           2891
State          2891
medAge         2891
malePop        2888
femPop         2888
totPop         2891
numVets        2878
foreignBorn    2878
avgHouseh      2891
stateCode      2891
Race           2891
Count          2891
dtype: int64


,City,State,medAge,malePop,femPop,totPop,numVets,foreignBorn,avgHouseh,stateCode,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


#### US Demographics
For my final dataframe same process as before but this time the data is pretty full, so we won't remove columns out of it. Went to drop any rows empty for my main column stateCode. Also changed the column names from the dataframe to be able to be passed. Dropped duplicates and updated null. 

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

![title](img/tables.png)

#### Why this format
I chose this model because I wanted to see the tables as is because these tables are detailed the way they are imported. Every table has there unique background in their subject and would use it for analysis and summary.

#### Format

For etl.py

1. create a spark session to be used
2. import the immigration data into our session
4. Check to see the data needs fixing
5. clean up the immigration data
6. write to s3 using spark emr
7. import airport data 

1. Import airplane data as a pandas dataframe
2. check the data for what to be cleaned
3. We will than clean up the data for use
4. check the data again
5. change the data type of the columns
6. bring into spark dataframe
7. create a view and select the data
8. write the data to s3 using EMR

1. import us demographics data into pandas dataframe
2. rename the columns to be able to import
2. check the data for what to be cleaned
3. We will than clean up the data for use
4. check the data again
5. change the data type of the columns
6. bring into spark dataframe
6. create a view and select the data
7. write the data to s3 using EMR



### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

The Pipeline is in etl.py please run it.
python etl.py

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

The quality check is also in the pipeline in etl.py
The data quality check im doing is data types for certain tables, check the dataframe if cleaning helped and count of the rows.

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Immigration Table
![title](img/immiDataTable.png)

#### Airplane Table
![title](img/airplane.png)

#### US Demographics Table
![title](img/usDemo.png)

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

I chose Apache Spark because it can handle alot of data and the the different types of formats its working with. Also, its great to use for an analysis tool. Using spark with EMR it can handle and process data fast but the # of workers it has or even how many cluster you have. Using spark would help customers reduce time in finding data and great results.

I think monthly would be great to update the data, because if your updating it weekly it might not be a difference or slight difference. Also, for updating constantly you can overload the cluster and increase cost. Monthly you will get a consistant coming in every month to support the use cases.

If we had to increase by one hundred, we should have an auto policy in aws that would automate itself to increase the number of workers and help the data overload.

To automate the dashboard I think we should set up a lambda function in aws to automate a data scrpt to fill in the dashbaord.

If many people are trying to get on, we should set up a RDB to handle the query of the users.